In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision import models, datasets
torch.__version__

'1.0.0.dev20190127'

In [2]:
%tensorboard --logdir log

UsageError: Line magic function `%tensorboard` not found.


In [3]:
from tensorboardX import SummaryWriter

In [4]:
cat_img = Image.open('data/cat_224x224.jpg')
cat_img.size

(224, 224)

In [5]:
cat_img_tensor = transforms.ToTensor()(cat_img)

In [6]:
writer = SummaryWriter(log_dir='./log', comment='cat image')
writer.add_image("cat", cat_img_tensor)
writer.close()

In [7]:
x = torch.FloatTensor([100])
y = torch.FloatTensor([500])

for epoch in range(100):
    x /= 1.5
    y /= 1.5
    loss = y-x
    with SummaryWriter(log_dir='./log', comment='train') as writer:
        writer.add_histogram('his/x', x, epoch)
        writer.add_histogram('his/y', y, epoch)
        writer.add_scalar('data/x', x, epoch)
        writer.add_scalar('data/y', y, epoch)
        writer.add_scalar('data/loss', loss, epoch)
        writer.add_scalars('data/data_group', {'x': x, 'y': y, 'loss': loss}, epoch)

In [8]:
vgg16 = models.vgg16(pretrained=True)
print(vgg16)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/liuzongxuan/.torch/models/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:47<00:00, 11644366.23it/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [9]:
vgg16_input = cat_img_tensor.view(1,3,224,224)

In [10]:
out = vgg16(vgg16_input)
_, preds = torch.max(out.data, 1)
label=preds.numpy()[0]
label

285

In [12]:
with SummaryWriter(log_dir='./log', comment='vgg16') as writer:
    writer.add_graph(vgg16, (vgg16_input,))